# LSTM. 8 featerow - mag, x, y, depth, dist, distance, plate, days_diff

In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd
SEED = 1337
tf.random.set_seed(SEED)
np.random.seed(SEED)

2023-11-22 08:44:40.426855: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-22 08:44:40.427383: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-22 08:44:40.430657: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-22 08:44:40.834919: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-22 08:44:44.548727: W tensorflow/compiler/

In [2]:
ds_train = tf.data.Dataset.load("../data/ds_final_train_old/")
ds_val = tf.data.Dataset.load("../data/ds_final_val_old/")
ds_test = tf.data.Dataset.load("../data/ds_final_test_old/")

2023-11-22 08:44:49.991946: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-11-22 08:44:51.147387: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-11-22 08:44:51.147491: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-11-22 08:44:51.163472: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-11-22 08:44:51.163804: I tensorflow/compile

In [3]:
BATCH_SIZE = 1024
BLOCK_SIZE = 64

In [4]:
def transform_element(features, context, labels):
    labels = tf.cast(labels, tf.int32)
    labels = tf.one_hot(labels, depth=2)
    return (features[:,:-2], features[:,-2], features[:,-1]), labels[0]

In [5]:
ds_train = ds_train.map(transform_element)
ds_val = ds_val.map(transform_element)
ds_test = ds_test.map(transform_element)

In [6]:
for x, y in ds_train.take(1):
    print(x[0].shape)
    print(x[1].shape)
    print(x[2].shape)
    print(y.shape)

(64, 6)
(64,)
(64,)
(2,)


In [7]:
ds_train = ds_train.batch(BATCH_SIZE).prefetch(tf.data.experimental.AUTOTUNE)
ds_val = ds_val.batch(BATCH_SIZE).prefetch(tf.data.experimental.AUTOTUNE)
ds_test = ds_test.batch(BATCH_SIZE).prefetch(tf.data.experimental.AUTOTUNE)

In [8]:
class MyModel(tf.keras.Model):
    def __init__(self, n_embed):
        super(MyModel, self).__init__()
        self.embed_dd = tf.keras.layers.Embedding(100, n_embed // 8)
        self.embed_plate = tf.keras.layers.Embedding(100, n_embed // 8)
        self.lstm_embed = tf.keras.layers.LSTM(n_embed // 8 * 6, return_sequences=True)
        self.lstm1 = tf.keras.layers.LSTM(n_embed, return_sequences=True)
        self.lstm2 = tf.keras.layers.LSTM(n_embed)
        self.dense1 = tf.keras.layers.Dense(n_embed, activation='relu')
        self.dense2 = tf.keras.layers.Dense(2, activation='softmax')
        self.conc = tf.keras.layers.Concatenate(axis=-1)

    def call(self, inputs):
        cont, plate, dd = inputs
        x1 = self.embed_dd(dd)
        x2 = self.embed_plate(plate)
        x3 = self.lstm_embed(cont)
        x = self.conc([x1, x2, x3])
        x = self.lstm1(x)
        x = self.lstm2(x)
        x = self.dense1(x)
        x = self.dense2(x)
        return x

In [9]:
model = MyModel(64)

In [10]:
model.build(input_shape=[(None, 64, 6), (None, 64), (None, 64)])
model.count_params()

82498

In [11]:
model.compile(optimizer=tf.keras.optimizers.Adam(),
                loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False),
                metrics=['accuracy', tf.keras.metrics.F1Score()])

In [12]:
history = model.fit(ds_train, epochs=10, validation_data=ds_val)

Epoch 1/10


2023-11-22 08:45:09.384896: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8904
Could not load symbol cublasGetSmCountTarget from libcublas.so.11. Error: /home/majkel/miniconda3/lib/libcublas.so.11: undefined symbol: cublasGetSmCountTarget
2023-11-22 08:45:11.379921: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7feecc065320 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-11-22 08:45:11.380027: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 2060 with Max-Q Design, Compute Capability 7.5
2023-11-22 08:45:11.440971: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-11-22 08:45:11.893644: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the proces

3227/3227 [==============================] - 294s 87ms/step - loss: 0.2055 - accuracy: 0.9383 - f1_score: 0.4915 - val_loss: 0.2207 - val_accuracy: 0.9377 - val_f1_score: 0.4839
Epoch 2/10
3227/3227 [==============================] - 317s 98ms/step - loss: 0.1620 - accuracy: 0.9469 - f1_score: 0.6645 - val_loss: 0.2185 - val_accuracy: 0.9397 - val_f1_score: 0.5643
Epoch 3/10
3227/3227 [==============================] - 304s 94ms/step - loss: 0.1585 - accuracy: 0.9486 - f1_score: 0.6843 - val_loss: 0.2183 - val_accuracy: 0.9429 - val_f1_score: 0.5872
Epoch 4/10
3227/3227 [==============================] - 281s 87ms/step - loss: 0.1552 - accuracy: 0.9496 - f1_score: 0.6839 - val_loss: 0.2098 - val_accuracy: 0.9418 - val_f1_score: 0.5700
Epoch 5/10
3227/3227 [==============================] - 275s 85ms/step - loss: 0.1535 - accuracy: 0.9497 - f1_score: 0.6864 - val_loss: 0.2107 - val_accuracy: 0.9416 - val_f1_score: 0.5654
Epoch 6/10
3227/3227 [==============================] - 282s 87ms/

In [13]:
model.evaluate(ds_test)

100/100 [==============================] - 11s 105ms/step - loss: 0.0891 - accuracy: 0.9799 - f1_score: 0.6845


[0.08910027891397476,
 0.9798869490623474,
 array([0.9897778 , 0.37929985], dtype=float32)]

In [14]:
model.save("../models/model_v2.keras")

In [15]:
# save history
history = pd.DataFrame.from_dict(history.history)
history.to_csv("../models/history_model_v2.csv", index=False)